# Exercise 2b: Numerical Simulation of the Advection Equation

## Learning Objectives

By completing this exercise, you will:
- Solve the advection equation numerically using finite differences
- Implement forward-in-time, forward-in-space (FTFS) scheme
- Apply periodic boundary conditions
- Compare numerical results with analytical solutions
- Understand the CFL condition for numerical stability

---

## Introduction

In this exercise, you will conduct your first numerical simulation of time evolution. We will solve the **advection equation**:

$$\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0  \tag{1}$$

where $a$ is a **constant**. In subsequent exercises, more general cases will be considered with $a$ no longer constant.

---

## Part 1: The Mathematical Problem ($a = \text{const}$)

### Problem Setup

Solve equation (1) numerically for $x \in [x_0, x_f]$ with:
- $x_0 = -2.6$, $x_f = 2.6$
- **Periodic boundary conditions**
- Initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2) \tag{2}$$

Start by representing this function in the given interval.

---

## Part 2: Spatial Derivative

### Discretization

Discretize the initial condition by subdividing the spatial domain into `nint = 64` equal intervals. The function is sampled with `nump = 65` points: $(u_0, u_1, u_2, \ldots, u_{nump-1})$ at equidistant locations $(x_0, x_1, x_2, \ldots, x_{nump-1})$.

Calculate the spatial derivative using **non-centered (forward) finite differences**:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i}}{\Delta x}  \tag{3}$$

> **Question:** Based on your experience from Exercise 1, what order of approximation can we expect when using **non-centered** finite differences?

---

## Part 3: Time Advance

### Time Discretization

To calculate the function at times later than $t = t_0$ (e.g., at $t_0 + \Delta t$), we discretize the time axis:

$$\left(\frac{\partial u}{\partial t}\right)_{\underset{t=t_0}{x=x_i}} \rightarrow \frac{u_i(t_0 + \Delta t)-u_i(t_0)}{\Delta t}\tag{4}$$

### The Update Formula

Using the differential equation (1) and expression (3), we obtain the **FTFS scheme**:

$$u_i(t_0+\Delta t) = u_i(t_0) - a \frac{u_{i+1}(t_0) - u_{i}(t_0)}{\Delta x}\Delta t  \tag{5}$$

### Task 3.1: Implement the Time Step

Using (5), calculate $u_i$ at time $t_0 + \Delta t$ at all points **excluding** the rightmost point ($x_{nump-1}$).

**Parameters:**
- $\Delta t = 0.98 \, \Delta x / |a|$ (CFL condition)
- $a = -1$

**Implementation:** Fill in the functions `step_adv_burgers` and `cfl_adv_burger` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_2.py).

---

## Part 4: Boundary Conditions

### Task 4.1: Apply Periodic Boundaries

To calculate the function at the rightmost point, apply **periodicity**:
$$u_{nump-1}(t_0 + \Delta t) = u_0(t_0 + \Delta t)$$

> **Tip:** Consider cutting ill-calculated (or missing) grid points and using `numpy.pad` with `mode='wrap'` to add periodic boundary conditions.

---

## Part 5: Time Evolution

### The General Time-Stepping Formula

Having calculated $u_i$ at time $t_0 + \Delta t$, we can continue stepping forward. In general, if $u^n_i$ denotes the value at position $x_i$ and time $n\Delta t$:

$$u_i^{n+1} = u_i^n - a \frac{u_{i+1}^n - u_i^n}{\Delta x}\Delta t  \tag{6}$$

Apply periodic boundary conditions (as in Part 4) at each timestep.

### Task 5.1: Implement the Evolution Loop

Fill in the function `evolv_adv_burgers` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_2.py).

### Task 5.2: Visualize the Evolution

Run many time steps to understand the mathematical nature of the solution.

> **Tip:** Use `matplotlib.animation` to visualize the time evolution.

---

## Part 6: Comparison with Exact Solution

### Task 6.1: Compare with Analytical Solution

From theory, we know the exact solution of eq. (1) for initial condition (2). 

**Steps:**
1. Plot the exact solution on top of the numerical one using a dashed line
2. Explain the mathematical behavior of the solution

> **Important:** The initial condition is eq. (2) in $[x_0, x_f]$ **with periodic boundary conditions**. This means the initial condition is an infinite repetition of function (2) placed side by side.

> **Hint:** Points starting within $(x_0, x_f)$ and moving with speed $a$ will exit the domain. You could use (not only solution):
> - The `%` (mod) operator to bring them back respecting periodicity, or
> - `numpy.pad` with ghost points at both ends of the domain

### Task 6.2: Set Up CI/CD Testing

Add a CI/CD pipeline to run this test and validate each push commit. This ensures that submitted changes don't break existing code.

**Implementation:** Fill in `./github/workflows/main.yml`

---

## Part 7: Resolution Study

### Task 7.1: Convergence with Resolution

Repeat the calculation increasing the number of spatial intervals by factors of 2. Verify that the numerical solution approaches the analytical solution.

> **Important:** Compare at the **same values of $x$ and $t$** for all resolutions. Choose a fixed time when the traveling function has crossed the domain several times.

---

## Hints

<details>
<summary><b>Hint 1: The CFL condition</b></summary>

The Courant-Friedrichs-Lewy (CFL) condition ensures stability:
$$\text{CFL} = \frac{|a| \Delta t}{\Delta x} \leq 1$$

Using $\Delta t = 0.98 \, \Delta x / |a|$ gives CFL = 0.98, safely below 1.
</details>

<details>
<summary><b>Hint 2: Periodic boundary with numpy.pad</b></summary>

```python
u_padded = np.pad(u[:-1], (0, 1), mode='wrap')
```
This wraps the first point to the last position.
</details>

<details>
<summary><b>Hint 3: Animation setup</b></summary>

```python
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
line, = ax.plot(x, u)

def update(frame):
    # Advance solution
    line.set_ydata(u_new)
    return line,

ani = FuncAnimation(fig, update, frames=nsteps)
```
</details>